In [3]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra

# Loading list of vertices, generating ids and dictionaries for conversion

with open("20230102-vertices.txt", "r") as vertex_file:

    vertex_lines = vertex_file.readlines()
    
vertex_counter = 0
vertices = []
dict_id_vertex = {}
dict_vertex_id = {}

for vertex_line in vertex_lines:
        
    vertex_line_stripped = vertex_line.rstrip()
    vertices.append(vertex_line_stripped)
    dict_id_vertex.update({str(vertex_counter): vertex_line_stripped})
    dict_vertex_id.update({vertex_line_stripped: vertex_counter})
    vertex_counter += 1
        
vertex_set = set(vertices)
        
# Loading and checking edges, building csr_matrix

with open("20240528-edges.txt", "r") as edge_file:

    edges_raw = edge_file.readlines()
    
line_counter = 0
empty_line_counter = 0
comment_line_counter = 0
edge_counter = 0
invalid_line_counter = 0
row = []
col = []
data = []    

for edge_raw in edges_raw:
        
    edge_stripped = edge_raw.rstrip()
    line_counter += 1
    edge_pieces = edge_stripped.split(",")
    
        
    if len(edge_pieces) == 1:
            
        if len(edge_stripped) == 0:
            empty_line_counter += 1
        elif edge_stripped[0] == "#":
            comment_line_counter += 1
        
    elif len(edge_pieces) == 2:
            
        if edge_pieces[0] in vertex_set and edge_pieces[1] in vertex_set:
            edge_counter += 1
            row.append(dict_vertex_id[edge_pieces[0]])
            col.append(dict_vertex_id[edge_pieces[1]])
            data.append(1)
        else:
            invalid_line_counter += 1
            print("LINE " + str(line_counter) + ": INVALID DATA !!!")
    
row =  np.array(row)
col =  np.array(col)
data = np.array(data)
    
edge_matrix = csr_matrix((data, (row, col)),
                         shape=(len(vertices), len(vertices)))

print(str(edge_matrix) + "\n")

print("Vertices: " + str(len(vertices)))
print("Lines read from edge file: " + str(line_counter))
print("Empty lines: " + str(empty_line_counter))
print("Comment lines: " + str(comment_line_counter))
print("Edges: " + str(edge_counter))
print("Invalid lines: " + str(invalid_line_counter))

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 533 stored elements and shape (204, 204)>
  Coords	Values
  (0, 27)	1
  (0, 41)	1
  (0, 48)	1
  (0, 102)	1
  (0, 127)	1
  (0, 181)	1
  (1, 34)	1
  (1, 107)	1
  (2, 21)	1
  (2, 145)	1
  (2, 157)	1
  (3, 33)	1
  (3, 88)	1
  (3, 140)	1
  (3, 154)	1
  (4, 75)	1
  (4, 150)	1
  (5, 34)	1
  (5, 46)	1
  (5, 61)	1
  (5, 203)	1
  (6, 84)	1
  (6, 85)	1
  (6, 139)	1
  (6, 140)	1
  :	:
  (162, 183)	1
  (162, 196)	1
  (163, 176)	1
  (165, 166)	1
  (165, 178)	1
  (168, 200)	1
  (169, 175)	1
  (169, 191)	1
  (169, 193)	1
  (169, 195)	1
  (171, 172)	1
  (171, 182)	1
  (171, 186)	1
  (172, 186)	1
  (172, 187)	1
  (174, 198)	1
  (175, 193)	1
  (175, 195)	1
  (175, 196)	1
  (178, 182)	1
  (178, 184)	1
  (181, 194)	1
  (182, 186)	1
  (183, 185)	1
  (193, 195)	1

Vertices: 204
Lines read from edge file: 776
Empty lines: 203
Comment lines: 40
Edges: 533
Invalid lines: 0


In [4]:
dist_matrix, predecessors = dijkstra(csgraph=edge_matrix,
                                     directed=False , return_predecessors=True)

print(str(dist_matrix) + "\n")
print(predecessors)

[[ 0.  8. 10. ...  2. 10.  8.]
 [ 8.  0.  6. ...  7.  4.  3.]
 [10.  6.  0. ...  9.  8.  5.]
 ...
 [ 2.  7.  9. ...  0.  9.  7.]
 [10.  4.  8. ...  9.  0.  4.]
 [ 8.  3.  5. ...  7.  4.  0.]]

[[-9999   107   157 ...   127    19   148]
 [  102 -9999   145 ...    25    19     5]
 [  102   107 -9999 ...    25    19   148]
 ...
 [  127   107   145 ... -9999    19   148]
 [  102   107   145 ...    25 -9999     5]
 [  102    34   145 ...    25    19 -9999]]


In [5]:
# Funktion route_dist zur Ermittlung der Knoten, über die
# der kürzeste Weg zwischen Start- und Endknoten führt
def route_dist (distances, predec, start_node, end_node):
    # Initialisierung von distance und route
    distance = int(0)
    route = [end_node]
    # Vom Zielknoten kommend wird der jeweilige Vorgänger-
    # knoten an route angehängt, solange der Startknoten
    # noch nicht erreicht ist
    while route[-1] != start_node:
        route.append(predec[start_node, route[-1]])
        distance += distances[route[-1],route[-2]]
    # Die Liste wird gedreht, damit die Knoten vom Start-
    # knoten zum Zielknoten angeordnet sind
    route.reverse()
    # Rückgabe eines dict mit der Knotenabfolge des kür-
    # zesten Wegs (route) und der zugehörigen Entfernung
    # (distance)    
    return {"route": route, "distance": distance}

def build_vertex_list(route):
    vertex_list = []
    for id in route:
        vertex_list.append(dict_id_vertex[str(id)])
    return vertex_list

In [6]:
result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Singen"], dict_vertex_id["Flensburg"])

build_vertex_list(result_dict["route"])

['Singen',
 'Villingen-Schwenningen',
 'Offenburg',
 'Baden-Baden',
 'Karlsruhe',
 'Heidelberg',
 'Darmstadt',
 'Frankfurt am Main',
 'Kassel',
 'Goslar',
 'Braunschweig',
 'Hannover',
 'Hamburg',
 'Neumünster',
 'Flensburg']

In [7]:
result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Kassel"], dict_vertex_id["Chemnitz"])

build_vertex_list(result_dict["route"])

['Kassel', 'Nordhausen', 'Naumburg', 'Gera', 'Chemnitz']

In [8]:
result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Schwäbisch Gmünd"], dict_vertex_id["Wittstock"])

build_vertex_list(result_dict["route"])

['Schwäbisch Gmünd',
 'Ansbach',
 'Nürnberg',
 'Bayreuth',
 'Hof',
 'Naumburg',
 'Dessau-Roßlau',
 'Berlin',
 'Wittstock']

In [9]:
# Exploration 04/04/2023: Introductory problem

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Stuttgart"], dict_vertex_id["Nürnberg"])

build_vertex_list(result_dict["route"])

['Stuttgart', 'Schwäbisch Gmünd', 'Ansbach', 'Nürnberg']

In [10]:
# Exploration 04/04/2023: Simple problem A

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Hof"], dict_vertex_id["Passau"])

build_vertex_list(result_dict["route"])

['Hof', 'Cham', 'Passau']

In [11]:
# Exploration 04/04/2023: Simple problem B

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Heilbronn"], dict_vertex_id["Nürnberg"])

build_vertex_list(result_dict["route"])

['Heilbronn', 'Ansbach', 'Nürnberg']

In [12]:
# Exploration 04/04/2023: Simple problem C

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Singen"], dict_vertex_id["Ravensburg"])

build_vertex_list(result_dict["route"])

['Singen', 'Sigmaringen', 'Ravensburg']

In [13]:
# Exploration 04/04/2023: Intermediate problem A

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Mannheim"], dict_vertex_id["Sigmaringen"])

build_vertex_list(result_dict["route"])

['Mannheim', 'Heilbronn', 'Stuttgart', 'Tübingen', 'Sigmaringen']

In [14]:
# Exploration 04/04/2023: Intermediate problem B

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Stuttgart"], dict_vertex_id["Amberg"])

build_vertex_list(result_dict["route"])

['Stuttgart', 'Schwäbisch Gmünd', 'Ansbach', 'Nürnberg', 'Amberg']

In [15]:
# Exploration 04/04/2023: Intermediate problem C

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Augsburg"], dict_vertex_id["Cham"])

build_vertex_list(result_dict["route"])

['Augsburg', 'München', 'Ingolstadt', 'Regensburg', 'Cham']

In [16]:
# Exploration 04/04/2023: Advanced problem A

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Freiburg"], dict_vertex_id["Freising"])

build_vertex_list(result_dict["route"])

['Freiburg',
 'Sigmaringen',
 'Biberach',
 'Memmingen',
 'Landsberg',
 'München',
 'Freising']

In [17]:
# Exploration 04/04/2023: Advanced problem B

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Karlsruhe"], dict_vertex_id["Coburg"])

build_vertex_list(result_dict["route"])

['Karlsruhe',
 'Heilbronn',
 'Ansbach',
 'Nürnberg',
 'Erlangen',
 'Bamberg',
 'Coburg']

In [18]:
# Exploration 04/04/2023: Advanced problem C

result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Heidelberg"], dict_vertex_id["Füssen"])

build_vertex_list(result_dict["route"])

['Heidelberg',
 'Heilbronn',
 'Heidenheim',
 'Ulm',
 'Memmingen',
 'Kempten',
 'Füssen']

In [19]:
result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Oldenburg"], dict_vertex_id["Freiburg"])

build_vertex_list(result_dict["route"])

['Oldenburg',
 'Cloppenburg',
 'Osnabrück',
 'Münster',
 'Soest',
 'Kassel',
 'Frankfurt am Main',
 'Darmstadt',
 'Heidelberg',
 'Karlsruhe',
 'Baden-Baden',
 'Offenburg',
 'Freiburg']

In [20]:
result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Oldenburg"], dict_vertex_id["Freiburg"])

build_vertex_list(result_dict["route"])

['Oldenburg',
 'Cloppenburg',
 'Osnabrück',
 'Münster',
 'Soest',
 'Kassel',
 'Frankfurt am Main',
 'Darmstadt',
 'Heidelberg',
 'Karlsruhe',
 'Baden-Baden',
 'Offenburg',
 'Freiburg']

In [21]:
result_dict = route_dist(dist_matrix, predecessors, dict_vertex_id["Ulm"], dict_vertex_id["Rostock"])

build_vertex_list(result_dict["route"])

['Ulm',
 'Heidenheim',
 'Ansbach',
 'Nürnberg',
 'Bayreuth',
 'Hof',
 'Naumburg',
 'Dessau-Roßlau',
 'Berlin',
 'Güstrow',
 'Rostock']